# Understanding Sustainability in the Job Market 
## Webscraping Indeed.com for "Sustainability" Jobs

## Part 1 - Extract Data
### Step 1 - Create Virtual Environment

- Create conda virtual environment for the project and its packages. The jupyter notebooks you work with will also be in this virtual environment.
- For reference: https://www.youtube.com/watch?v=Ro9l0eapoJU
- Download the following packages in the virtual environment
- Open a jupyter notebook in the virtual environment

In [1]:
import pandas as pd
import numpy as np
import lxml
import requests
from bs4 import BeautifulSoup as bs
import datetime
import time
from collections import defaultdict
import re

### Step 2 - Studying the URL and the DOM
- We have three different pages to scrape
    - Job Results Page
    - Job Descriptions Page
    - Company Information Page
- Check if the website allows you to webscrape
- Study the URL. Take note of two things:
    - What parts of the URL stays the same?
    - What parts change when you change the search/job/company?
- Study the DOM. Take note of:
    - Where is the information you are interested in? Note the tags they are nested in.
        - Does this change every so often? Then you might want to scrape another website.
        - More information here: https://www.bestproxyreviews.com/how-to-scrape-linkedin-using-proxies/#:~:text=LinkedIn%20is%20very%2C%20very%20against%20scraping%20of%20any%20kind.
        - Also here: https://medium.com/nerd-for-tech/linked-in-web-scraper-using-selenium-15189959b3ba
        - And here: https://medium.com/geekculture/bypass-scraping-websites-that-has-css-class-names-change-frequently-d4877ecd6d8f
    - How do the tags and attributes change for different searches/jobs/companies? 

### For this project
- URL of interest: https://www.indeed.com/jobs?q=sustainability&l=California
    - When you click the next page: https://www.indeed.com/jobs?q=sustainability&l=California&start=10
Increases by increments of 10
- When you view a specific job: https://www.indeed.com/viewjob?jk=178c02cc700910ee
- When you visit a company website: https://www.indeed.com/cmp/Stantec

### Step 3A - Scraping the Job Results Page 

In [3]:
# create jj - list to store each search result (dictionary)
jj = []
# counter to count # of job results scraped
counter = 0

# number of pages to scrape. This is used to calculate num_jobs, which is the format required for URL
pages = 1
num_jobs = pages * 10

for j in range(0, num_jobs, 10):
    
    print(f'Starting scrape for page {j//10}/{num_jobs//10}.')
    
    # ADD the position, location search terms here
    position, location = 'sustainability', 'United+States'
    
    # OPTIONAL: MODIFY user agent with yours here.
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

    y = requests.get(f'https://www.indeed.com/jobs?q={position}&l={location}&start={str(j)}', headers = header)
    
    # OPTIONAL: MODIFY time.sleep(#) and if/else conditions here. This was done to prevent CAPTCHA
    if j % 2 == 0 and j % 5 == 0 and j < 50:
        time.sleep(7)
    elif j % 2 != 0 and j % 5 == 0 and j < 50:
        time.sleep(4)
    elif j % 3 != 0 and j < 50:
        time.sleep(3)
    
    time.sleep(3)
    
    # sp is the html version of the website you requested with y.
    # Think you are getting CAPTCHAs? Check by print(sp) - will return html with CAPTCHA 
    sp = bs(y.text, 'lxml')
    
    # soup is finding the html of the table that contains the results of interest. 
    # MODIFY as necessary for your needs.
    # Think you are getting CAPTCHAs? Check by print(soup) - will return []
    soup = sp.find('table',{"id":"pageContent"})

    

    # for each div tag with the class = "jobsearch-SerpJobCard"
    # MODIFY as necessary for your needs.
    for ii in soup.find_all(name='div',attrs={"class":"jobsearch-SerpJobCard"}):
        
        # find job title, company name, location
        # MODIFY with the infos you are looking for
        job_title= ii.find('a',{"class":"jobtitle"})['title']
        company_name= ii.find('span',{"class":"company"}).text.strip()
        location= ii.find('span',{"class":"location"})
        
        # if location is not found, check if there is a div that contains class: location
        # MODIFY as needed.
        if location:
            location=location.text.strip()
        else:
            location=ii.find('div',{"class":"location"})
            location=location.text.strip()
                
        # extract the job key, which will lead us to the specific job's page
        k=bs(str(ii), 'lxml')
        jk=k.find(name='div', attrs={"class":"jobsearch-SerpJobCard"})
        jobkey=jk['data-jk']
    
        # create dictionary for the job result, with the values found above
        job_dict={'job_title': job_title,
                    'company_name': company_name,
                    'location': location,
                    'job_key': jobkey}
        
        # add count for job result scraped
        counter += 1
        
        # append job dictionary to the list of jobs - jj
        jj.append(job_dict)

print(f'Appended {counter} entries to jj list')

Starting scrape for page 0/1.
Appended 15 entries to jj list


In [4]:
#Should be same as counter above
print(len(jj))
#Check first 3 job entries
jj[:3] 

15


[{'job_title': 'Emergency Preparedness Trainee, Security Safety & Sustainability - Summer/Fall 2021',
  'company_name': 'Sony Pictures Entertainment, Inc.',
  'location': 'Culver City, CA 90232 (Lucerne/Higuera area)',
  'job_key': '83f882108ba070bc'},
 {'job_title': 'Junior Sustainability Engineer',
  'company_name': 'Belkin',
  'location': 'Playa Vista, CA 90094',
  'job_key': '389725f97041bb80'},
 {'job_title': 'Environmental Specialist Level II',
  'company_name': 'Worley',
  'location': 'Monrovia, CA',
  'job_key': '2de7b3821880203c'}]

In [5]:
## convert jj into a basic_jobs data frame. 
basic_jobs = pd.DataFrame(jj)

# export basic_jobs into csv to save what you scraped!
# OPTIONAL: change name of file as needed.
basic_jobs.to_csv('0jobkeys.csv')

### Step 3B - Scraping the Job Description Page

In [6]:
# we will grab the csv we created from scraping the job results page to grab the job keys

# read the csv with job results
# CHANGE file name as needed
jj = pd.read_csv('0jobkeys.csv')

# create dictionary (key= column name, values= list of each index) with the data frame jj
jj.to_dict('list')

# assign the list of job keys to jk
jk= (jj['job_key'])

In [7]:
# create empty dictionary for job descriptions. key will be index as in jk, value will be string (job description)
descriptions = {}

# for each item in jk
# OPTIONAL: change # of descriptions you want to scrape as needed.
for i in range(len(jk)):

    # OPTIONAL: MODIFY user agent with yours here.
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
    jp = requests.get(f"https://www.indeed.com/viewjob?jk={jk[i]}", headers = header)

    # OPTIONAL: MODIFY/REMOVE print statements as desired. These were to check the progress of the scraper.
    print(f"Request for {i}/{len(jk)-1} jobs")
    print(f"https://www.indeed.com/viewjob?jk={jk[i]}\n")
    
    if i % 2 == 0 and i % 5 == 0 and i < 5:
        time.sleep(2)
    elif i % 2 != 0 and i % 5 == 0 and i < 5:
        time.sleep(4)
    elif i % 3 != 0 and i < 50:
        time.sleep(3)
    
    time.sleep(3)
    
    # jp_soup is the html text that was pulled from request jp
    # Think you are getting CAPTCHAs? Check by print(jp) - will return html with captcha 
    jp_soup= bs(jp.text,'html.parser')
    
    # raw_desc is finding the html for "div" tags with class attribute "jobsearch..." - this is where the job description is located!!
    # MODIFY as necessary for your needs. (again, we are looking for job description text)
    # Think you are getting CAPTCHAs? Check by print(raw_desc) - will return []
    raw_desc = (jp_soup.find("div",{"class":"jobsearch-jobDescriptionText"}))
    
    # if raw_desc is None, then we don't have a job description. For this specific scraper, this means we have a captcha
    # scraper will break here if it comes across a captcha
    if raw_desc is None:
        print("NONE TYPE DETECTED")
        break
    
    # try grabbing text from the job description and strip it (remove /n)
    # if it raises an attribute error (ex. HTML formatting not correct), then assume it's blank.
    # add to dictionary. key = job_key, value = job description as string
    try:
        descriptions[jk[i]] =(raw_desc.text.strip())
    except AttributeError:
        descriptions[jk[i]] = ""

print(f'Extracted {len(descriptions)} job descriptions.')

Request for 0/15 jobs
https://www.indeed.com/viewjob?jk=83f882108ba070bc

Request for 1/15 jobs
https://www.indeed.com/viewjob?jk=389725f97041bb80

Request for 2/15 jobs
https://www.indeed.com/viewjob?jk=2de7b3821880203c

Request for 3/15 jobs
https://www.indeed.com/viewjob?jk=23f873fd0d5e663e

Request for 4/15 jobs
https://www.indeed.com/viewjob?jk=e862369e52ce2651

Request for 5/15 jobs
https://www.indeed.com/viewjob?jk=f1b5042ee03a92cd

Request for 6/15 jobs
https://www.indeed.com/viewjob?jk=d9c58f73a2edd6d0

Request for 7/15 jobs
https://www.indeed.com/viewjob?jk=2215f107fd1c0ba2

Request for 8/15 jobs
https://www.indeed.com/viewjob?jk=398ff78866abe65d

Request for 9/15 jobs
https://www.indeed.com/viewjob?jk=d03123777b27b108

Request for 10/15 jobs
https://www.indeed.com/viewjob?jk=037f3323b9192c81

Request for 11/15 jobs
https://www.indeed.com/viewjob?jk=cf4d4f9e0009297a

Request for 12/15 jobs
https://www.indeed.com/viewjob?jk=cc7e05bf63d6cb1d

Request for 13/15 jobs
https://www.

NameError: name 'descriptions_list' is not defined

In [10]:
# convert the dictionary of job descriptions into a dataframe. the index is the job_key.
desc_jobs = pd.DataFrame.from_dict(descriptions, orient='index', columns = ['job_desc'])

# make new column, name = index, values = job_key. 
desc_jobs.reset_index(inplace = True)

# check first 3 entries of job descriptions
desc_jobs[:3]

,index,job_desc
0,83f882108ba070bc,The Sony Pictures Entertainment Professional T...
1,389725f97041bb80,The Junior Sustainability Engineer (JSE) prima...
2,2de7b3821880203c,"48,000 people. 49 countries. Over 120 office l..."


In [11]:
# export job descriptions to csv. this will save what you scraped!
# MODIFY file name as desired.
desc_jobs.to_csv('0jobdescriptions.csv')

### Step 3C - Scraping the Company Info Page

In [12]:
# we will grab the csv we created from scraping the job results page to grab the company names

# read the csv with job results
# CHANGE file name as needed
job_results = pd.read_csv('0jobkeys.csv')
job_results

,Unnamed: 0,job_title,company_name,location,job_key
0,0,"Emergency Preparedness Trainee, Security Safet...","Sony Pictures Entertainment, Inc.","Culver City, CA 90232 (Lucerne/Higuera area)",83f882108ba070bc
1,1,Junior Sustainability Engineer,Belkin,"Playa Vista, CA 90094",389725f97041bb80
2,2,Environmental Specialist Level II,Worley,"Monrovia, CA",2de7b3821880203c
3,3,"Data Analyst, Sustainability",Netflix,"Los Angeles, CA",23f873fd0d5e663e
4,4,Sustainability Innovation Consultant,"Amazon Web Services, Inc.",Remote,e862369e52ce2651
5,5,Sustainability Analyst,Andersen Corporation,"Oak Park Heights, MN",f1b5042ee03a92cd
6,6,Environmental Sustainability Manager,The Walt Disney Company (Corporate),"Burbank, CA",d9c58f73a2edd6d0
7,7,"Coordinator, Procurement Sustainability",Danone,"Broomfield, CO",2215f107fd1c0ba2
8,8,Applied Scientist (Sustainability),Descartes Labs,Remote,398ff78866abe65d
9,9,Global Sustainability Specialist,"NuVasive, Inc.","San Diego, CA",d03123777b27b108


In [13]:
# this scraper will scrape company info for each unique company

# create array of unique values in 'company_name' column of job_results
company_unique = job_results['company_name'].unique()

# create a list. values = dictionary with company info, as ordered in company_unique.
cc = []

# counter for # of companies scraped
counter = 0

# counter for # of companies that were not scraped (company name not formatted correctly in URL)
pagenotfound = 0

# counter for # of companies with no company info
noinfo = 0

# for each item in company_unique
for c in range(len(company_unique)):
    
    # create default company_info dictionary
    cd = {'ceo': '-1',
          'founded': '-1',
          'size': '-1',
          'revenue': '-1',
          'industry': '-1'       
         }
    
    # grab company name from company_unique using index
    cname = company_unique[c]
    
    # formatting cname for URL
    # extract name before comma
    fcname = re.split(r',+',cname)
    
    # replace space with -
    fcname = re.sub(r'\s','-',fcname[0])
    
    # request for URL
    cr = requests.get(f'https://www.indeed.com/cmp/{fcname}')
    
    # added variable sleep timers to avoid captcha
    # OPTIONAL: change if/else conditions and time.sleep(#)
    if c % 2 == 0 and c % 5 == 0 and c < 10:
        time.sleep(7)
    elif c % 2 != 0 and c % 5 == 0 and c < 10:
        time.sleep(4)
    elif c% 3 != 0 and c < 10:
        time.sleep(3)
    elif c % 2 == 0 and c % 5 == 0 and c >= 10:
        time.sleep(2)
    elif c % 2 != 0 and c % 5 == 0 and c >= 10:
        time.sleep(9)
    elif c % 3 != 0 and c >= 10:
        time.sleep(5)
    
    time.sleep(5)
    
    # if the URL request was successful
    if cr.status_code == 200:
        
        # sp is the html of the page that we asked for in request cr
        sp = bs(cr.text, 'lxml')
        
        # soup is the html of the ul with class = eu40a1w0, which contains the info we are looking for
        soup = sp.find('ul', {"class": 'eu4oa1w0'})
        
        # print statement to confirm URL request was sent successfully
        # OPTIONAL: REMOVE if desired
        print(f'{c} {fcname} URL REQUEST SUCCESSFUL')
        
        # if soup is None, we do not have any info on the company OR we hit a captcha. can't distinguish.
        if soup is None:
            
            print("\tNONE TYPE DETECTED")
            noinfo += 1
            
            # append default dictionary 
            cc.append(cd)
            
            # go to next index of for loop
            continue
        
        # for each "li" item in soup
        for ii in soup.find_all(name="li"):
            
            # extract info
            # OPTIONAL: MODIFY as needed
            if ii.get('data-testid') == 'companyInfo-ceo':
                cd['ceo'] = (re.sub('CEO', '', ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-founded':
                cd['founded'] = (re.sub('Founded', '', ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-employee':
                cd['size'] = (re.sub('Company size', '', ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-revenue':
                cd['revenue'] = (re.sub('Revenue', '',ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-industry':
                cd['industry'] = (re.sub('Industry', '', ii.text)) 
                
        print("\tInfo Extract SUCCESS")
     
    # cr.status code 404 is page not found (incorrect URL formatting)
    else:
        print(f'{c} {fcname} PAGE NOT FOUND')
        pagenotfound +=1
    
    
    counter += 1
    cc.append(cd)
    
print(f'Appended {counter} companies to cc list')
print(f'Page Not Found for {pagenotfound} companies')
print(f'No info found for {noinfo} companies') 

0 Sony-Pictures-Entertainment URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
1 Belkin URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
2 Worley URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
3 Netflix URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
4 Amazon-Web-Services URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
5 Andersen-Corporation URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
6 The-Walt-Disney-Company-(Corporate) URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
7 Danone URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
8 Descartes-Labs URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
9 NuVasive URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
10 ERM-Group URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
11 Tetra-Tech URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
12 University-of-Nebraska-Medical-Center URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
13 APCO-Worldwide URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
14 Seattle-Housing-Authority URL REQUEST SUCCESSFUL
	Info Extract SUCCESS
Appended 15 companies to cc list

In [15]:
# convert the dictionary of company info into a dataframe. the index is as ordered in company_unique
comp_info = pd.DataFrame(cc)

# make new column, name = index, values = job_key. 
comp_info['company_name'] = company_unique

# check first 3 entries of job descriptions
comp_info[:3]

,ceo,founded,size,revenue,industry,company_name
0,-1,1982,"5,001 to 10,000",$5B to $10B (USD),Media & Communication,"Sony Pictures Entertainment, Inc."
1,-1,1983,"501 to 1,000",$1B to $5B (USD),Manufacturing,Belkin
2,-1,1971,"more than 10,000",$1B to $5B (USD),"Energy, Mining & Utilities",Worley
3,Reed Hastings,1997,"5,001 to 10,000",more than $10B (USD),Media & Communication,Netflix
4,-1,-1,-1,-1,Information Technology,"Amazon Web Services, Inc."
5,-1,1903,"5,001 to 10,000",$1B to $5B (USD),Commercial Equipment Services,Andersen Corporation
6,Robert Iger,1923,"more than 10,000",more than $10B (USD),Healthcare,The Walt Disney Company (Corporate)
7,-1,1919,"more than 10,000",more than $10B (USD),Manufacturing,Danone
8,-1,2014,11 to 50,less than $1M (USD),-1,Descartes Labs
9,-1,1997,"1001 to 5,000",$500M to $1B (USD),Hospitals & Health Clinics,"NuVasive, Inc."


In [16]:
# export company information to csv. this will save what you scraped!
# MODIFY file name as desired.
comp_info.to_csv('0companyinfo.csv')

## Part 2 - Transform Data (preparing the scraped data for us to use for analysis)
### Step 1 - Merge Data

In [17]:
# We will merge job keys with job descriptions. Then merge that with company info.

# call each of the data frames
# MODIFY file names as needed
job_keys = pd.read_csv('0jobkeys.csv')
job_desc = pd.read_csv('0jobdescriptions.csv')
comp_info = pd.read_csv('0companyinfo.csv')

In [21]:
#drop unnamed: 0 column
job_keys.drop('Unnamed: 0', axis=1, inplace=True)

job_keys

,job_title,company_name,location,job_key
0,"Emergency Preparedness Trainee, Security Safet...","Sony Pictures Entertainment, Inc.","Culver City, CA 90232 (Lucerne/Higuera area)",83f882108ba070bc
1,Junior Sustainability Engineer,Belkin,"Playa Vista, CA 90094",389725f97041bb80
2,Environmental Specialist Level II,Worley,"Monrovia, CA",2de7b3821880203c
3,"Data Analyst, Sustainability",Netflix,"Los Angeles, CA",23f873fd0d5e663e
4,Sustainability Innovation Consultant,"Amazon Web Services, Inc.",Remote,e862369e52ce2651
5,Sustainability Analyst,Andersen Corporation,"Oak Park Heights, MN",f1b5042ee03a92cd
6,Environmental Sustainability Manager,The Walt Disney Company (Corporate),"Burbank, CA",d9c58f73a2edd6d0
7,"Coordinator, Procurement Sustainability",Danone,"Broomfield, CO",2215f107fd1c0ba2
8,Applied Scientist (Sustainability),Descartes Labs,Remote,398ff78866abe65d
9,Global Sustainability Specialist,"NuVasive, Inc.","San Diego, CA",d03123777b27b108


In [22]:
#drop unnamed: 0 column
job_desc.drop('Unnamed: 0', axis=1, inplace=True)

job_desc

,index,job_desc
0,83f882108ba070bc,The Sony Pictures Entertainment Professional T...
1,389725f97041bb80,The Junior Sustainability Engineer (JSE) prima...
2,2de7b3821880203c,"48,000 people. 49 countries. Over 120 office l..."
3,23f873fd0d5e663e,"Los Gatos, California\nLos Angeles, California..."
4,e862369e52ce2651,"7+ years of experience in consulting, solution..."
5,f1b5042ee03a92cd,Description\n\nSustainability Analyst\nA key t...
6,d9c58f73a2edd6d0,Job Summary:\nEnvironmental sustainability at ...
7,2215f107fd1c0ba2,About the job\nAbout the Job:\nPosition Descri...
8,398ff78866abe65d,The Company\nDescartes Labs helps the world's ...
9,d03123777b27b108,Job Summary and Primary Responsibilities\nThis...


In [23]:
#drop unnamed: 0 column
comp_info.drop('Unnamed: 0', axis=1, inplace=True)

comp_info

,ceo,founded,size,revenue,industry,company_name
0,-1,1982,"5,001 to 10,000",$5B to $10B (USD),Media & Communication,"Sony Pictures Entertainment, Inc."
1,-1,1983,"501 to 1,000",$1B to $5B (USD),Manufacturing,Belkin
2,-1,1971,"more than 10,000",$1B to $5B (USD),"Energy, Mining & Utilities",Worley
3,Reed Hastings,1997,"5,001 to 10,000",more than $10B (USD),Media & Communication,Netflix
4,-1,-1,-1,-1,Information Technology,"Amazon Web Services, Inc."
5,-1,1903,"5,001 to 10,000",$1B to $5B (USD),Commercial Equipment Services,Andersen Corporation
6,Robert Iger,1923,"more than 10,000",more than $10B (USD),Healthcare,The Walt Disney Company (Corporate)
7,-1,1919,"more than 10,000",more than $10B (USD),Manufacturing,Danone
8,-1,2014,11 to 50,less than $1M (USD),-1,Descartes Labs
9,-1,1997,"1001 to 5,000",$500M to $1B (USD),Hospitals & Health Clinics,"NuVasive, Inc."


In [ ]:
#if you scraped in increments, you may need to append data tables to each other (add extra rows to the same columns)
#you can do that with .append

#combined = firsthalf.append(secondhalf, ignore_index = True)

### Merging Job Results with Job Description

In [24]:
# Merge on job_key
job_info = job_keys.merge(job_desc, how = "inner", left_on="job_key", right_on="index")
job_info

,job_title,company_name,location,job_key,index,job_desc
0,"Emergency Preparedness Trainee, Security Safet...","Sony Pictures Entertainment, Inc.","Culver City, CA 90232 (Lucerne/Higuera area)",83f882108ba070bc,83f882108ba070bc,The Sony Pictures Entertainment Professional T...
1,Junior Sustainability Engineer,Belkin,"Playa Vista, CA 90094",389725f97041bb80,389725f97041bb80,The Junior Sustainability Engineer (JSE) prima...
2,Environmental Specialist Level II,Worley,"Monrovia, CA",2de7b3821880203c,2de7b3821880203c,"48,000 people. 49 countries. Over 120 office l..."
3,"Data Analyst, Sustainability",Netflix,"Los Angeles, CA",23f873fd0d5e663e,23f873fd0d5e663e,"Los Gatos, California\nLos Angeles, California..."
4,Sustainability Innovation Consultant,"Amazon Web Services, Inc.",Remote,e862369e52ce2651,e862369e52ce2651,"7+ years of experience in consulting, solution..."
5,Sustainability Analyst,Andersen Corporation,"Oak Park Heights, MN",f1b5042ee03a92cd,f1b5042ee03a92cd,Description\n\nSustainability Analyst\nA key t...
6,Environmental Sustainability Manager,The Walt Disney Company (Corporate),"Burbank, CA",d9c58f73a2edd6d0,d9c58f73a2edd6d0,Job Summary:\nEnvironmental sustainability at ...
7,"Coordinator, Procurement Sustainability",Danone,"Broomfield, CO",2215f107fd1c0ba2,2215f107fd1c0ba2,About the job\nAbout the Job:\nPosition Descri...
8,Applied Scientist (Sustainability),Descartes Labs,Remote,398ff78866abe65d,398ff78866abe65d,The Company\nDescartes Labs helps the world's ...
9,Global Sustainability Specialist,"NuVasive, Inc.","San Diego, CA",d03123777b27b108,d03123777b27b108,Job Summary and Primary Responsibilities\nThis...


In [25]:
# Drop the 'index' column that was added during merge
job_info.drop('index', axis = 1, inplace = True)
job_info

,job_title,company_name,location,job_key,job_desc
0,"Emergency Preparedness Trainee, Security Safet...","Sony Pictures Entertainment, Inc.","Culver City, CA 90232 (Lucerne/Higuera area)",83f882108ba070bc,The Sony Pictures Entertainment Professional T...
1,Junior Sustainability Engineer,Belkin,"Playa Vista, CA 90094",389725f97041bb80,The Junior Sustainability Engineer (JSE) prima...
2,Environmental Specialist Level II,Worley,"Monrovia, CA",2de7b3821880203c,"48,000 people. 49 countries. Over 120 office l..."
3,"Data Analyst, Sustainability",Netflix,"Los Angeles, CA",23f873fd0d5e663e,"Los Gatos, California\nLos Angeles, California..."
4,Sustainability Innovation Consultant,"Amazon Web Services, Inc.",Remote,e862369e52ce2651,"7+ years of experience in consulting, solution..."
5,Sustainability Analyst,Andersen Corporation,"Oak Park Heights, MN",f1b5042ee03a92cd,Description\n\nSustainability Analyst\nA key t...
6,Environmental Sustainability Manager,The Walt Disney Company (Corporate),"Burbank, CA",d9c58f73a2edd6d0,Job Summary:\nEnvironmental sustainability at ...
7,"Coordinator, Procurement Sustainability",Danone,"Broomfield, CO",2215f107fd1c0ba2,About the job\nAbout the Job:\nPosition Descri...
8,Applied Scientist (Sustainability),Descartes Labs,Remote,398ff78866abe65d,The Company\nDescartes Labs helps the world's ...
9,Global Sustainability Specialist,"NuVasive, Inc.","San Diego, CA",d03123777b27b108,Job Summary and Primary Responsibilities\nThis...


### Adding Company Info to Job Info

In [26]:
# we need to create the company info columns to job_info.
companies = job_info['company_name']

ci = ['ceo','founded','size','revenue','industry']

for col in ci:
    job_info[col] = ['-1' for x in range(len(companies))]

job_info

,job_title,company_name,location,job_key,job_desc,ceo,founded,size,revenue,industry
0,"Emergency Preparedness Trainee, Security Safet...","Sony Pictures Entertainment, Inc.","Culver City, CA 90232 (Lucerne/Higuera area)",83f882108ba070bc,The Sony Pictures Entertainment Professional T...,-1,-1,-1,-1,-1
1,Junior Sustainability Engineer,Belkin,"Playa Vista, CA 90094",389725f97041bb80,The Junior Sustainability Engineer (JSE) prima...,-1,-1,-1,-1,-1
2,Environmental Specialist Level II,Worley,"Monrovia, CA",2de7b3821880203c,"48,000 people. 49 countries. Over 120 office l...",-1,-1,-1,-1,-1
3,"Data Analyst, Sustainability",Netflix,"Los Angeles, CA",23f873fd0d5e663e,"Los Gatos, California\nLos Angeles, California...",-1,-1,-1,-1,-1
4,Sustainability Innovation Consultant,"Amazon Web Services, Inc.",Remote,e862369e52ce2651,"7+ years of experience in consulting, solution...",-1,-1,-1,-1,-1
5,Sustainability Analyst,Andersen Corporation,"Oak Park Heights, MN",f1b5042ee03a92cd,Description\n\nSustainability Analyst\nA key t...,-1,-1,-1,-1,-1
6,Environmental Sustainability Manager,The Walt Disney Company (Corporate),"Burbank, CA",d9c58f73a2edd6d0,Job Summary:\nEnvironmental sustainability at ...,-1,-1,-1,-1,-1
7,"Coordinator, Procurement Sustainability",Danone,"Broomfield, CO",2215f107fd1c0ba2,About the job\nAbout the Job:\nPosition Descri...,-1,-1,-1,-1,-1
8,Applied Scientist (Sustainability),Descartes Labs,Remote,398ff78866abe65d,The Company\nDescartes Labs helps the world's ...,-1,-1,-1,-1,-1
9,Global Sustainability Specialist,"NuVasive, Inc.","San Diego, CA",d03123777b27b108,Job Summary and Primary Responsibilities\nThis...,-1,-1,-1,-1,-1


In [29]:
# populating the company info columns using the comp_info data frame, if the company is in the comp_info dataframe

for company in companies:
    for index in ci:
        if company in comp_info['company_name']:
            thevalue = comp_info.loc[comp_info['company_name'] == company, index]
            (job_info.loc[job_info['company_name'] == company, index]) = thevalue

job_info

,job_title,company_name,location,job_key,job_desc,ceo,founded,size,revenue,industry
0,"Emergency Preparedness Trainee, Security Safet...","Sony Pictures Entertainment, Inc.","Culver City, CA 90232 (Lucerne/Higuera area)",83f882108ba070bc,The Sony Pictures Entertainment Professional T...,-1,1982,"5,001 to 10,000",$5B to $10B (USD),Media & Communication
1,Junior Sustainability Engineer,Belkin,"Playa Vista, CA 90094",389725f97041bb80,The Junior Sustainability Engineer (JSE) prima...,-1,1983,"501 to 1,000",$1B to $5B (USD),Manufacturing
2,Environmental Specialist Level II,Worley,"Monrovia, CA",2de7b3821880203c,"48,000 people. 49 countries. Over 120 office l...",-1,1971,"more than 10,000",$1B to $5B (USD),"Energy, Mining & Utilities"
3,"Data Analyst, Sustainability",Netflix,"Los Angeles, CA",23f873fd0d5e663e,"Los Gatos, California\nLos Angeles, California...",Reed Hastings,1997,"5,001 to 10,000",more than $10B (USD),Media & Communication
4,Sustainability Innovation Consultant,"Amazon Web Services, Inc.",Remote,e862369e52ce2651,"7+ years of experience in consulting, solution...",-1,-1,-1,-1,Information Technology
5,Sustainability Analyst,Andersen Corporation,"Oak Park Heights, MN",f1b5042ee03a92cd,Description\n\nSustainability Analyst\nA key t...,-1,1903,"5,001 to 10,000",$1B to $5B (USD),Commercial Equipment Services
6,Environmental Sustainability Manager,The Walt Disney Company (Corporate),"Burbank, CA",d9c58f73a2edd6d0,Job Summary:\nEnvironmental sustainability at ...,Robert Iger,1923,"more than 10,000",more than $10B (USD),Healthcare
7,"Coordinator, Procurement Sustainability",Danone,"Broomfield, CO",2215f107fd1c0ba2,About the job\nAbout the Job:\nPosition Descri...,-1,1919,"more than 10,000",more than $10B (USD),Manufacturing
8,Applied Scientist (Sustainability),Descartes Labs,Remote,398ff78866abe65d,The Company\nDescartes Labs helps the world's ...,-1,2014,11 to 50,less than $1M (USD),-1
9,Global Sustainability Specialist,"NuVasive, Inc.","San Diego, CA",d03123777b27b108,Job Summary and Primary Responsibilities\nThis...,-1,1997,"1001 to 5,000",$500M to $1B (USD),Hospitals & Health Clinics


In [33]:
# export to csv to save the dataframes we merged!
job_info.to_csv('0combined_info.csv', index = False)

## Part 3 - Load Data

In [34]:
load_data = pd.read_csv('0combined_info.csv')
load_data

# data is ready to be explored!

,job_title,company_name,location,job_key,job_desc,ceo,founded,size,revenue,industry
0,"Emergency Preparedness Trainee, Security Safet...","Sony Pictures Entertainment, Inc.","Culver City, CA 90232 (Lucerne/Higuera area)",83f882108ba070bc,The Sony Pictures Entertainment Professional T...,-1,1982,"5,001 to 10,000",$5B to $10B (USD),Media & Communication
1,Junior Sustainability Engineer,Belkin,"Playa Vista, CA 90094",389725f97041bb80,The Junior Sustainability Engineer (JSE) prima...,-1,1983,"501 to 1,000",$1B to $5B (USD),Manufacturing
2,Environmental Specialist Level II,Worley,"Monrovia, CA",2de7b3821880203c,"48,000 people. 49 countries. Over 120 office l...",-1,1971,"more than 10,000",$1B to $5B (USD),"Energy, Mining & Utilities"
3,"Data Analyst, Sustainability",Netflix,"Los Angeles, CA",23f873fd0d5e663e,"Los Gatos, California\nLos Angeles, California...",Reed Hastings,1997,"5,001 to 10,000",more than $10B (USD),Media & Communication
4,Sustainability Innovation Consultant,"Amazon Web Services, Inc.",Remote,e862369e52ce2651,"7+ years of experience in consulting, solution...",-1,-1,-1,-1,Information Technology
5,Sustainability Analyst,Andersen Corporation,"Oak Park Heights, MN",f1b5042ee03a92cd,Description\n\nSustainability Analyst\nA key t...,-1,1903,"5,001 to 10,000",$1B to $5B (USD),Commercial Equipment Services
6,Environmental Sustainability Manager,The Walt Disney Company (Corporate),"Burbank, CA",d9c58f73a2edd6d0,Job Summary:\nEnvironmental sustainability at ...,Robert Iger,1923,"more than 10,000",more than $10B (USD),Healthcare
7,"Coordinator, Procurement Sustainability",Danone,"Broomfield, CO",2215f107fd1c0ba2,About the job\nAbout the Job:\nPosition Descri...,-1,1919,"more than 10,000",more than $10B (USD),Manufacturing
8,Applied Scientist (Sustainability),Descartes Labs,Remote,398ff78866abe65d,The Company\nDescartes Labs helps the world's ...,-1,2014,11 to 50,less than $1M (USD),-1
9,Global Sustainability Specialist,"NuVasive, Inc.","San Diego, CA",d03123777b27b108,Job Summary and Primary Responsibilities\nThis...,-1,1997,"1001 to 5,000",$500M to $1B (USD),Hospitals & Health Clinics
